<a href="https://colab.research.google.com/github/kimdanny/COMP0189-practical/blob/main/Week-08/week8-problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP0189: Applied Artificial Intelligence
## Week 8 (Reinforcement Learning)

In this notebook you will learn how to implement from scratch the Q-Learning algorithm for deterministic Markov
Decision Process and how to use the Deep Q-Learning algorithm using the Pytorch library.

Imagine we want to train an impostor from
[Among Us](https://en.wikipedia.org/wiki/Among_Us) to make it capable of finding the closest vent to
hide himself. To simplify this task we assume that the impostor moves in a grid world.

We can model the impostor in this world as an agent that implements 4 actions,
move up, move down, move right and move left,
and we can model the environment as a grid world where the agent gets rewarded only once it has achieved its goal,
it has found the vent.

## The Environment

If in supervised learning, we would usually spend most of our time collecting and cleaning the data, in RL this
time is spent in designing the environment. Following we design a `GridWorld` class.

### Acknowledgements
This notebook is adapted from [CEGE0004: Machine Learning for Data Science](https://github.com/aldolipani/CEGE0004) (week 8 RL practical), written by [Dr. Aldo Lipani](https://aldolipani.com) (aldo.lipani@ucl.ac.uk), an Asst. Prof. in Machine Learning at the University College London (UCL).

In [ ]:
# Getting the Among Us images
! wget https://raw.githubusercontent.com/kimdanny/COMP0189-practical/main/Week-08/imgs/agent.png
! wget https://raw.githubusercontent.com/kimdanny/COMP0189-practical/main/Week-08/imgs/target.png

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

class GridWold:

    def __init__(self, n, init_pos=(0, 0), target_pos=None):
        """
        A simple environment
        :param n: The size of the world.
        :param init_pos: The initial position of the agent. The agent start at the top-left corner by default.
        :param target_pos: The position of the goal. The goals is set to the bottom-right corner by default.
        """
        self.n = n
        self.init_pos = init_pos
        self.pos = init_pos
        if target_pos is None:
            self.target_pos = (n - 1, n - 1)
        else:
            self.target_pos = target_pos

    def reset(self, pos=None):
        """
        This method resets the environment.
        :param pos: If we set the position, the agent will start not from the default position by from the
        position set. This is useful when we will add some randomness in the training.
        :return:
        """
        if pos is None:
            self.pos = self.init_pos
        else:
            self.pos = pos
        return self.pos

    def get_available_actions(self):
        """
        This method returns the set of available actions the agent can perform given its current position.
        :return: The set of available actions.
        """
        # if the agent has reached the vent, then no actions are available.
        if self.pos == self.target_pos:
            res = {}
        else:
            # all actions
            res = {'up', 'down', 'right', 'left'}
            # remove actions if the agent is at the edges of the world.
            if self.pos[0] == 0:
                res.discard('left')
            elif self.pos[0] == self.n - 1:
                res.discard('right')
            if self.pos[1] == 0:
                res.discard('up')
            elif self.pos[1] == self.n - 1:
                res.discard('down')
        return res

    def step(self, action: str):
        """
        This method executes an action and changes the state of the agent. It returns the new position of the agent and
        the reward received.
        :param action: The action to execute.
        :return: The position and reward.
        """
        # executes an action
        if action == 'left':
            self.pos = (self.pos[0] - 1, self.pos[1])
        elif action == 'right':
            self.pos = (self.pos[0] + 1, self.pos[1])
        elif action == 'down':
            self.pos = (self.pos[0], self.pos[1] + 1)
        elif action == 'up':
            self.pos = (self.pos[0], self.pos[1] - 1)

        # if the agent has achieved the goal, it returns a reward of 100, zero otherwise.
        if self.pos == self.target_pos:
            return self.pos, 100
        else:
            return self.pos, 0

    def render(self):
        """
        This method renders the environment and the agent.
        """
        n = self.n
        agent = OffsetImage(plt.imread('agent.png'), zoom=1 / (2 * n))
        target = OffsetImage(plt.imread('target.png'), zoom=10 / (2 * n))

        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(1, 1, 1)
        for j in range(n + 1):
            ax.axvline(x=j / n)
            ax.axhline(y=j / n)

        ab_target = AnnotationBbox(target,
                                   xy=((self.target_pos[0] + 0.5) / n, 1 - (self.target_pos[1] + 0.7) / n),
                                   frameon=False)
        ax.add_artist(ab_target)

        ab_agent = AnnotationBbox(agent,
                                  xy=((self.pos[0] + 0.5) / n, 1 - (self.pos[1] + 0.5) / n),
                                  frameon=False)
        ax.add_artist(ab_agent)
        plt.show()

Let's now instantiate a gird-world of size 5 and render it. If we use the default parameters, you should see the
impostor on the top-left corner, and the vent on the bottom-right corner.

In [ ]:
env = GridWold(5)
env.render()

# A Random Policy

We will now simulate a random policy. An agent that navigates the world performing random actions. This agent will
execute actions for a given number of times. While executing the actions
we will also count the number of times the agent has achieved its goal (the number of episodes).

## Overview
A Random Policy is a **basic decision-making strategy** in reinforcement learning where an agent selects actions uniformly at random. This approach serves as:

- Baseline comparison for learning algorithms
- Pure exploration mechanism

- Simple starting point for policy analysis


## Mathematical Formulation
For discrete action space with $n$ possible actions:
$$\pi(a|s) = \frac{1}{n}\ \ \forall a \in \mathcal{A}(s)$$

Note that we are slowing down the execution of this policy to make this policy visible; We add a 1ms delay at every
agent step.

In [ ]:
import time
import random
from tqdm import tqdm
from IPython import display

**Task 1. Run your random policy and render the image each step**

In [ ]:
# the number of actions the agent will execute in total
total_steps = 1000

episodes = 0
for n in tqdm(range(total_steps)):
    # delay for display
    time.sleep(0.1)
    display.clear_output(wait=True)
    available_actions = env.get_available_actions()
    # Implement and run a random policy
    ### YOUR CODE GOES BELOW ###
    None
    
    env.render()

# Q-Learning

We will now implement the Q-learning algorithm to learn the Q-values.

## Overview
Q-Learning is a **model-free reinforcement learning** algorithm that enables agents to learn optimal policies through interaction with an environment. It uses a Q-table to store expected future rewards for each state-action pair $(s,a)$.

## Key Components

- **Q-Table**: Matrix storing $Q(s,a)$ values
- **Learning Rate ($\alpha$)**: Controls update magnitude (0 < α ≤ 1)

- **Discount Factor ($\gamma$)**: Values future rewards (0 ≤ γ ≤ 1)
- **Epsilon-Greedy**: Balances exploration/exploitation

## Bellman Update Equation
$$Q(s,a) \leftarrow Q(s,a) + \alpha\left[r + \gamma \max_{a'} Q(s',a') - Q(s,a)\right]$$

## Algorithm Steps

1. Initialize Q-table with zeros
2. Observe current state $s$

3. Choose action $a$ (epsilon-greedy)
4. Receive reward $r$, observe new state $s'$

5. Update Q-value using Bellman equation
6. Repeat until convergence

## Instruction
In the following code, we simplify the equation by setting the Learning Rate ($\alpha$) to 1, therefore $$Q(s,a) \leftarrow r + \gamma \max_{a'} Q(s',a')$$


**Task 2. Implement and train with Q-Learning**

In [ ]:
# discount rate
l = 0.9
# the number of actions the agent will execute in total
total_steps = 1000
# q-values
q_table = {}

# reset env
pos = env.reset()

# render env
env.render()
time.sleep(0.1)
display.clear_output(wait=True)

episodes = 0
for n in tqdm(range(total_steps)):
    # delay for display
    time.sleep(0.1)
    display.clear_output(wait=True)
    # get available actions
    available_actions = env.get_available_actions()
    # Implement and run Q-Learning
    ### YOUR CODE GOES BELOW ###
    None
    
    env.render()

**Task 3. Now that we have trained the agent we can test it. Compare the performance of this agent against the random policy.**

In [ ]:
# the number of actions the agent will execute in total
total_steps = 1000

# reset env
pos = env.reset()
for n in tqdm(range(total_steps)):
    time.sleep(0.1)
    display.clear_output(wait=True)
    # get available actions
    available_actions = env.get_available_actions()
    ### YOUR CODE GOES BELOW ###
    None

    env.render()

## An example of Ready-Made Environments (MiniGrid)

Here I provide you with an example of a ready-made environment where you can test RL algorithms.
These environments are defined by the [MiniGrid](https://github.com/maximecb/gym-minigrid).

Before executing the next code, please make sure that this package is correctly installed.

In [ ]:
!pip install gym-minigrid==1.0.2

In [ ]:
from gym_minigrid.wrappers import *

env = gym.make('MiniGrid-Empty-16x16-v0')

This is how the environment looks like at the beginning. We have a grid world with an agent in the top-left corner
that can navigate this empty room. On the bottom-right corner we have the target location.

In [ ]:
# You might have to run this cell twice if you face error.
plt.figure(figsize=(9, 9))
env.reset()
plt.imshow(env.render(mode='rgb_array'))
plt.show()

We can access the actions available to the agent as follows:

In [ ]:
for action in list(env.actions):
    print(action)

In [ ]:
# You might have to run this cell twice if you face error.

from tqdm import tqdm
total_steps = 100

for n in tqdm(range(total_steps)):
    time.sleep(0.1)
    display.clear_output(wait=True)
    plt.imshow(env.render(mode='rgb_array'))
    plt.show()
    action = env.action_space.sample()
    _, reward, _, _ = env.step(action)

    print(f'Iteration: {n + 1} of Episodes {episodes}')
    print(f'Action taken: {action}')
    print(f'Reward: {reward}')

## The Same Cartpole Example in Pytorch

https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

[Run in Colab](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/9da0471a9eeb2351a488cd4b44fc6bbf/reinforcement_q_learning.ipynb)